In [1]:
import os
import requests
import csv
import re
import datetime, calendar
from datetime import timedelta
from bs4 import BeautifulSoup
import pandas as pd



In [2]:
# Funció per fer el dataframe d'un mes
def __audiencia_del_mes(anyo, mes):
    """DOCSTRING_DE_FUNCION"""
    
    data = pd.DataFrame(columns=('Audiencia', 'Cadena', 'Dia', 'Mes','Año'))
    
    year = anyo
    month = mes

    num_days = calendar.monthrange(year, month)[1]
    days = [datetime.date(year, month, day) for day in range(1, num_days+1)]
    for day in days: 
        fecha = day.strftime('%Y-%m-%d')
        url = 'https://www.formulatv.com/audiencias/'+fecha
        req = requests.get(url, timeout=10)
        soup = BeautifulSoup(req.text, "lxml")

        # Busquem els ul
        ul = soup.h1.find_next_sibling("ul")
        cadenas1 = (" ".join([img['alt'] for img in ul.find_all('img', alt=True)]))
        cadenas1 = re.sub(re.compile('logo'),"",cadenas1)

        # Ho passem a llista
        cadenas1 = cadenas1.split(" ")
        # Treiem el percentatges, és a dir, el text de cada li
        # Escollim posar un espai com a separador

        audiencias1 = ul.get_text(' ', strip=True)
        # Ho passem a llista
        audiencias1 = audiencias1.split(" ")
        
        audiencias = pd.DataFrame(audiencias1, columns=['Audiencia'])
        cadenas = pd.DataFrame(cadenas1, columns=['Cadena'])

        df = pd.concat([cadenas, audiencias], axis=1, sort=False) 
        df["Dia"] = day.strftime('%d')
        df["Mes"] = day.strftime('%m')
        df["Año"] = day.strftime('%Y')
        
        data = pd.concat([data, df]) 
    
    return data

In [3]:
data_1Q_2019 = pd.DataFrame(columns=('Audiencia', 'Cadena', 'Dia', 'Mes','Año'))

year=2019

# 1er Quatrimestre
for new_month in range(1, 5):
    df = __audiencia_del_mes(year, new_month)
    data_1Q_2019  = pd.concat([data_1Q_2019, df]) 


C:\Users\lula-\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\lula-\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [7]:
# Reset Index

data_1Q_2019 = data_1Q_2019.reset_index()
# Fitxer

local_url = r'C:\Users\lula-\Documents\UOC Data Science\Tipologia\export_1Q_year_'+str(year)+'.csv'
data_1Q_2019.to_csv (local_url, index = False, header=True)

      index Audiencia   Año          Cadena Dia Mes
0         0     15,9%  2019             la1  01  01
1         1     14,2%  2019         antena3  01  01
2         2      9,1%  2019       telecinco  01  01
3         3      5,3%  2019          cuatro  01  01
4         4      5,0%  2019         lasexta  01  01
...     ...       ...   ...             ...  ..  ..
3090     21      0,8%  2019  gol-television  30  04
3091     22      0,5%  2019             ten  30  04
3092     23      0,4%  2019         bemadtv  30  04
3093     24      0,3%  2019     teledeporte  30  04
3094     25      0,2%  2019  real-madrid-tv  30  04

[3095 rows x 6 columns]
